In [ ]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json, yaml
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

BARLOC = os.path.join(lib.gamedir, "KH2", "ard", "eh23.ard")

lib.openkh.bar_extract(BARLOC, os.path.join(os.getcwd(),"eh23"))

In [15]:
name = "Barbossa"

msn = None
useadds = True
adds = [
#     {"ObjectId": 358}
]
sp_dragon = yaml.load(open("m_50.spawn"))
sp_buildings = yaml.load(open("b_40.spawn"))
sp_actors = yaml.load(open("b_99.spawn"))
import shutil

sp_dragon[0]["Entities"][0]["ObjectId"] = 0

for spid in sp_buildings:
    for ent in spid["Entities"]:
        if ent["ObjectId"] == 1766:
            ent["ObjectId"] = 0

new_sp = get_new_sp(sp_actors, name, msn, useadds, adds)
yaml.dump(sp_actors, open("b_99_new.spawn.yml", "w"))
yaml.dump(sp_buildings, open("b_40_new.spawn.yml", "w"))
yaml.dump(sp_dragon, open("m_50_new.spawn.yml", "w"))
lib.openkh.spawnpoint_build(os.path.join(os.getcwd(), "m_50_new.spawn.yml"), os.path.join(os.getcwd(), "eh23", "m_50.spawn"))
lib.openkh.spawnpoint_build(os.path.join(os.getcwd(), "b_40_new.spawn.yml"), os.path.join(os.getcwd(), "eh23", "b_40.spawn"))
lib.openkh.spawnpoint_build(os.path.join(os.getcwd(), "b_99_new.spawn.yml"), os.path.join(os.getcwd(), "eh23", "b_99.spawn"))
lib.openkh.bar_build(os.path.join(os.getcwd(),"eh23", "eh23.ard.json"), BARLOC)

<ipython-input-15-5ab89e94526a>:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sp_dragon = yaml.load(open("m_50.spawn"))
<ipython-input-15-5ab89e94526a>:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sp_buildings = yaml.load(open("b_40.spawn"))
<ipython-input-15-5ab89e94526a>:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sp_actors = yaml.load(open("b_99.spawn"))


['repoint', '-o', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\eh23\\m_50.spawn', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\m_50_new.spawn.yml']
(None, None)
['repoint', '-o', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\eh23\\b_40.spawn', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\b_40_new.spawn.yml']
(None, None)
['repoint', '-o', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\eh23\\b_99.spawn', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\b_99_new.spawn.yml']
(None, None)
['pack', '-o', 'C:\\Users\\Arcade\\Desktop\\pcsx2-ex\\KH2\\KH2\\ard\\eh23.ard', 'C:\\Users\\Arcade\\Desktop\\git\\kh2lib\\examples\\Using kh2lib\\sephiroth replacement testing\\eh23\\eh23.ard.json']
(None,

In [16]:


def get_new_sp(sp, name, msn, useadds, adds):
    from khbr.randomizer import KingdomHearts2
    kh2en = KingdomHearts2().enemy_records
    if name in kh2en:
        obj = kh2en[name]
    else:
        raise Exception("Name not found")    
    
    msn = msn or obj["msn"] if "msn" in obj else None

    
    
    sora = sp[0]["Entities"][0]
    sora["PositionY"] = 14940
    bossz = float(sora["PositionZ"])
    sora["PositionZ"] = float(sp[0]["Entities"][2]["PositionZ"])
    
    riku = sp[0]["Entities"][1]
    riku["PositionY"] = 14940
    riku["PositionZ"] = float(sp[0]["Entities"][2]["PositionZ"])
    
    
    seph = sp[0]["Entities"][2]
    seph["ObjectId"] = obj["obj_id"]
    seph["PositionY"] = 14940
    seph["PositionZ"] = bossz
    seph["Argument1"] = obj["vars"][0]
    seph["Argument2"] = obj["vars"][1]

    new_sp = sp

    if useadds:
        for add in adds:
            newent = dict(seph)
            for k in add:
                newent[k] = add[k]
            sp[0]["Entities"].append(newent)
    elif "adds" in obj:
        for add in obj["adds"]:
            addobj = kh2en[add["name"]]
            newent = dict(seph)
            newent["ObjectId"] = addobj["obj_id"]
            newent["Argument1"] = addobj["vars"][0]
            newent["Argument2"] = addobj["vars"][1]

            sp[0]["Entities"].append(newent)
            
    msn_base_dir = os.path.join(lib.gamedir, "KH2", "msn", "jp")
    if msn:
        shutil.copy(os.path.join(msn_base_dir, msn+".bar"),os.path.join(msn_base_dir, "HB01_MS601.bar"))

    return new_sp